In [15]:
import os
import sys
import requests
from cachetools import cached, TTLCache
import time
import pandas as pd
import tqdm

g_token=''

@cached(cache=TTLCache(maxsize=1, ttl=36000))
def get_token():
    # step 1:  using username='30007', password='2A4nFXhP' 
    ##         on https://mid.meddra.org/account/register for apikey
    # step 2:  apply for grant
    cmd = '''
    curl \
        --request POST \
        --data "grant_type=password"  \
        --data "username={username}"  --data "password={apikey}" \
        --data "scope=meddraapi" \
        --data "client_id=mspclient" \
        "https://mid.meddra.org/connect/token"
    '''.format(username='30007', apikey='4Lj68iPSy3jjti7hK2hM')
    os.system(cmd)
    # ref: https://cloud.tencent.com/developer/article/1908468
    return auth['access_token']
    # {"access_token":"6acb65aaaa2eee65530de8e2521606b10051a5b8a5451bdf1227eb704bda080d","expires_in":36000,"token_type":"Bearer"}

def get_token2():
    return "767067280cb04954b8ac16493aef7bee88a859d64d6c8fc6a257231127b85215"  #8-12 19:00

def get_soc(row):
    ## (1) search pt name for pt-code
    # considering sleep 3s
    
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer {}'.format(get_token2()),
        'Content-Type': 'text/json',
    }

    pt_name = row['pt']   # "heart valve operation"

    data = '{"bview":"SOC","rsview":"release","language":"English","stype":1,"addlangs":[],"filters":[],"version":24,"searchterms":[{"searchtype":0,"searchterm":"' \
           + pt_name + '","searchlogic":0}],"kana":false,"idiacritical":true,"synonym":false,"contains":false,"soc":false,"hlgt":false,"hlt":false,"pt":true,"llt":false,"smq":false,"skip":0,"take":0,"separator":2}'
    
    try:
        response = requests.post('https://gxpmapiprod.meddra.org/api/search', headers=headers, data=data)
        #print(response.json())
        ##(2) detail: search soc name & code using pt_code
        pt_code = response.json()[0]['pcode']
    
        data = '{"bview":"SOC","rsview":"release","code":' + str(pt_code) + \
               ',"pcode":0,"syncode":0,"lltcode":0,"ptcode":0,"hltcode":0,"hlgtcode":0,"soccode":0,"smqcode":0,"type":"PT","addlangs":[],"rtype":"P","lang":"English","ver":24,"kana":false,"separator":2}'
    
        response = requests.post('https://gxpmapiprod.meddra.org/api/detail', headers=headers, data=data)
        soc_name = ''
        soc_code = -1
    
        for item in response.json()['mds']:
            #print(item)
            if item['primarysocfg'] == 'Y':
                soc_name = item['socname']
                soc_code = item['soccode']
                           
        ## finally output results using dataframes
        #return {'pt': pt_name, 'pt_code': pt_code, 'soc_name': soc_name, 'soc_code': soc_code}
        return pd.Series([pt_name, pt_code, soc_name, soc_code], index=['pt_name', 'pt_code', 'soc_name', 'soc_code'])

    except Exception as e:
        print("error for: " + pt_name)
    finally:
        pass


In [16]:
#get_token()
conn.close()

NameError: name 'conn' is not defined

In [ ]:
conn = pymysql.connect(
        host='localhost',
        port=int(3306),
        user="root",
        passwd='0921051011Liu',
        db="faers",
        charset='utf8mb4')

# 去除异常字符
sql = '''   
   SELECT 
        replace(coalesce(t2.pt,'not_found_pt'),'\\r', '') as pt,
        COUNT(distinct t1.CASEID) AS caseid_cnt,
        COUNT(distinct t1.PRIMARYID) AS primaryid_cnt
    FROM faers.ribo_drug_usage t1
    left join faers.faers_reac t2
    on t1.primaryid = t2.primaryid and lower(t1.role_cod) = 'ps'
    group by 1
    order by 1;
'''
df_pt_stat = pd.read_sql_query(sql, conn)

conn.close()

In [ ]:
import tqdm
from tqdm import tqdm
tqdm.pandas(desc='apply')

df_pt_stat=pd.read_csv('./ribo_pt.csv')
pt_soc_mapping = df_pt_stat.progress_apply(get_soc, axis=1)
#pt_soc_mapping = df_pt_stat.apply(get_soc, axis=1)

#df_pt_stat[ df_pt_stat['caseid_cnt'] != df_pt_stat['primaryid_cnt']]

apply:   0%|          | 2/3223 [02:20<62:59:13, 70.40s/it]

error for: abdominal adhesions


apply:   9%|▊         | 281/3223 [15:31<1:31:13,  1.86s/it]

error for: benign pancreatic hyperenzymaemia


apply:   9%|▉         | 283/3223 [15:34<1:22:29,  1.68s/it]

...

apply:  14%|█▍        | 447/3223 [22:43<1:36:40,  2.09s/it]

error for: brain fog


apply:  16%|█▌        | 513/3223 [25:52<1:35:09,  2.11s/it]

error for: buttock claudication


apply:  19%|█▉        | 628/3223 [31:24<4:08:22,  5.74s/it]

In [ ]:
error for: anal inflammation
error for: anaphylactic shock
error for: aneurysm
error for: anger
error for: aspartate aminotransferase abnormal
error for: aspartate aminotransferase increased
error for: benign pancreatic hyperenzymaemia
error for: bile duct obstruction
error for: biliary tract disorder
error for: bilirubin conjugated abnormal
error for: bilirubin conjugated increased
error for: biopsy breast
error for: blood viscosity decreased
error for: body mass index abnormal
error for: body mass index decreased
error for: body mass index increased
error for: bone demineralisation
error for: bone pain
error for: bone scan abnormal
error for: brain fog
error for: brca1 gene mutation
error for: brca2 gene mutation
error for: breast calcifications
error for: breast cancer
error for: breast cancer female
error for: buttock claudication
error for: calcification metastatic
error for: calcinosis
error for: calciphylaxis
error for: calcium deficiency
error for: cancer fatigue
error for: cerebral ischaemia
error for: cerebral microangiopathy
error for: cholecystitis acute
error for: cholecystitis chronic
error for: cholelithiasis
error for: congenital cystic kidney disease
error for: congestive cardiomyopathy
error for: conjunctival haemorrhage
error for: conjunctivitis
error for: connective tissue disorder
error for: contraindicated product administered
error for: contraindicated product prescribed
error for: craniocerebral injury
error for: creatinine renal clearance decreased
error for: creatinine renal clearance increased
error for: crepitations
error for: creutzfeldt-jakob disease
error for: device related sepsis
error for: dilated cardiomyopathy
error for: drug dose omission
error for: drug effect incomplete